In [5]:
from data_utils import *
from evaluate import *
from dataset import TLDataset
from transformers import T5ForConditionalGeneration,AutoTokenizer, AutoConfig
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import argparse
import logging
import torch
from multiprocessing import cpu_count
from finetuner import T5FineTuner
from ray_lightning import RayStrategy
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from tqdm import tqdm
from pytorch_lightning import loggers as pl_loggers
import ray
import ast
import os
import gc


output_path_dir = "/home/salzubi_umass_edu/experiments/"
train_bsz = 8
lr = 3e-4
train_fraction = 1
model_name = "google/flan-t5-base"
num_gpus = torch.cuda.device_count()
num_cpus = cpu_count()

args_dict = dict(
    # file_name = file_name,
    # output_dir=output_path_dir, # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_seq_length=512,
    train_fraction= train_fraction,
    lr=lr,
    weight_decay=0.0,
    scheduler_factor = .1,
    use_gpu = True if num_gpus > 0 else False,
    train_batch_size=train_bsz,
    eval_batch_size=32,
    num_train_steps=10000000,
    es_patience = 4,
    val_check_interval = .25,
    dropout = .2,
    n_gpu=num_gpus,
    cpu_per_device=num_cpus,
    task = "classification",
)

args = argparse.Namespace(**args_dict)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

In [6]:
from data_utils import *
from evaluate import *

model = load_ckpt(args, f"../experiments/mocha_tests/SRC_task_[50, 33, 40, 32, 31, 51, 39, 52, 53, 57, 64, 54, 55]___TRG_task___None/trial_2/SRC_task_[50, 33, 40, 32, 31, 51, 39, 52, 53, 57, 64, 54, 55]___TRG_task___None___trial_2-v1.ckpt")

In [7]:
test = pd.read_csv("../experiments/mocha_tests/processed_sample_2.csv")
test_data = TLDataset(test, tokenizer)
test_loader = DataLoader(test_data, batch_size=2, shuffle=False, num_workers=32, drop_last=False)

/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
evaluated_batch = evaluate_batch(test_loader, model, tokenizer, out_dir="../experiments/mocha_tests/sample_2_preds.csv")

25it [02:51,  6.87s/it]Traceback (most recent call last):
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
Traceback (most recent call last):
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/work/salz

	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 673, in _rmtree_safe_fd
    os.unlink(entry.name, dir_fd=topfd)
OSError: [Errno 16] Device or resource busy: '.nfs6687f651badb9ee9000004ce'
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 675, in _rmtree_safe_fd
    onerror(os.unlink, fullname, sys.exc_info())
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
OSError: [E

	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 718, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/work/salzubi_umas

...

  File "/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/shutil.py", line 673, in _rmtree_safe_fd
    os.unlink(entry.name, dir_fd=topfd)


	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


OSError: [Errno 16] Device or resource busy: '.nfse47ae5159f42ea62000004ca'
OSError: [Errno 16] Device or resource busy: '.nfs572b1547ab0853ec000004c8'
OSError: [Errno 16] Device or resource busy: '.nfsb84561efd1d80552000004bf'


	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


25it [02:51,  6.87s/it]
